## **ANN Group A Assignment_7: Implement Artificial Neural Network training process in Python by using Forward Propagation, Back Propagation.**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.Wij = np.random.uniform(-0.5, 0.5, (self.input_size, self.hidden_size)) # Weights from input to hidden layer
        self.Vjk = np.random.uniform(-0.5, 0.5, (self.hidden_size, self.output_size)) # Weights from hidden to output layer
        
        self.bias_j = np.random.uniform(-0.5, 0.5, (1, self.hidden_size)) # hidden layer bias
        self.bias_k = np.random.uniform(-0.5, 0.5, (1, self.output_size)) # output  layer bias
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
     
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_prop(self, x):
        self.hidden_layer_input = np.dot(x, self.Wij) + self.bias_j # weighted sum  of inputs to the hidden layer
        self.hidden_layer_output = self.sigmoid(self.hidden_layer_input) # output  of the hidden layer
        
        self.output_layer_input = np.dot(self.hidden_layer_output, self.Vjk) + self.bias_k
        self.output_layer_output = self.sigmoid(self.output_layer_input)
        
        return self.output_layer_output
    
    def backprop(self, x, y, output):
        error = y - output
        delta_output = error * self.sigmoid_derivative(output)
        
        hidden_layer_error = delta_output.dot(self.Vjk.T)
        hidden_layer_delta = hidden_layer_error * self.sigmoid_derivative(self.hidden_layer_output)
        
        # weight adjustment between hidden and output layers
        self.Vjk += self.hidden_layer_output.T.dot(delta_output) * self.learning_rate
         
       # calculate new bias value for the output layer
        self.bias_k += np.sum(delta_output) * self.learning_rate
         
        # weight adjustment between input and hidden layers
        self.Wij += x.T.dot(hidden_layer_delta) * self.learning_rate
        
        # calculate new bias value for the hidden layer
        self.bias_j += np.sum(hidden_layer_delta) * self.learning_rate
        
    def train(self, x, y, epochs):
        for epoch in range(epochs):
            output = self.forward_prop(x)
            self.backprop(x, y, output)
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}: Error {np.mean(np.abs(y - output))}')

In [3]:
df = pd.read_csv('Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df['Species'] = df['Species'].replace({'Iris-setosa': 1, 'Iris-virginica': 0, 'Iris-versicolor': 0})
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1
3,4,4.6,3.1,1.5,0.2,1
4,5,5.0,3.6,1.4,0.2,1


In [5]:
arr1 = np.array([df['SepalLengthCm'], df['SepalWidthCm'], df['PetalLengthCm'], df['PetalWidthCm']]).T
arr2 = np.array(df['Species'])
arr1[0:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(arr1, arr2, test_size=0.2, random_state=30)

# One-hot encode target labels for training
encoder = OneHotEncoder(categories='auto')
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
print(y_train_onehot[:5])
# Training
input_size = X_train.shape[1]
hidden_size = 8
output_size = y_train_onehot.shape[1]
learning_rate = 0.1
epochs = 10000

obj_ANN = ANN(input_size, hidden_size, output_size, learning_rate)
obj_ANN.train(X_train, y_train_onehot, epochs)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Epoch 0: Error 0.46695894646049607
Epoch 1000: Error 0.00768570499328992
Epoch 2000: Error 0.005350570920920106
Epoch 3000: Error 0.004337798878980958
Epoch 4000: Error 0.0037393308939756838
Epoch 5000: Error 0.0033326175356397343
Epoch 6000: Error 0.003032797759667247
Epoch 7000: Error 0.0027995160980900436
Epoch 8000: Error 0.0026107429817408333
Epoch 9000: Error 0.0024532097919008947


In [25]:
# Testing
print("\nPredictions after training:\n")
print("Input Features\tSL | SW | PL | PW\t    Predicted Class\t Actual Class")
for i in range(len(X_test)):
    predicted = obj_ANN.forward_prop(X_test[i])
    predicted_class = np.argmax(predicted)
    actual_class = y_test[i]
    print(f"\t\t{X_test[i]}\t\t    {predicted_class}\t\t     {actual_class}") # setosa: 1 , versicolor: 0 or virginica: 0


Predictions after training:

Input Features	SL | SW | PL | PW	    Predicted Class	 Actual Class
		[5.4 3.4 1.7 0.2]		    1		     1
		[5.4 3.9 1.7 0.4]		    1		     1
		[4.6 3.1 1.5 0.2]		    1		     1
		[5.8 2.7 5.1 1.9]		    0		     0
		[5.8 2.7 3.9 1.2]		    0		     0
		[5.5 2.4 3.8 1.1]		    0		     0
		[6.3 2.7 4.9 1.8]		    0		     0
		[6.7 3.  5.2 2.3]		    0		     0
		[6.4 2.9 4.3 1.3]		    0		     0
		[7.3 2.9 6.3 1.8]		    0		     0
		[4.4 3.  1.3 0.2]		    1		     1
		[6.3 2.9 5.6 1.8]		    0		     0
		[5.  2.3 3.3 1. ]		    0		     0
		[5.7 2.8 4.1 1.3]		    0		     0
		[4.4 2.9 1.4 0.2]		    1		     1
		[6.2 2.9 4.3 1.3]		    0		     0
		[5.2 4.1 1.5 0.1]		    1		     1
		[4.9 3.  1.4 0.2]		    1		     1
		[4.8 3.4 1.9 0.2]		    1		     1
		[6.3 2.5 4.9 1.5]		    0		     0
		[6.3 2.8 5.1 1.5]		    0		     0
		[4.4 3.2 1.3 0.2]		    1		     1
		[5.1 3.5 1.4 0.2]		    1		     1
		[5.  3.2 1.2 0.2]		    1		     1
		[7.6 3.  6.6 2.1]		    0		     0
		[6.4 2.8 5.6 2.2]		    0		

In [21]:
# Calculate accuracy
correct_count = 0
total = len(x_test)

for i in range(total):
    predicted = obj_ANN.forward_prop(x_test[i])
    predicted_class = np.argmax(predicted)
    if predicted_class == y_test[i]:
        correct_count += 1

accuracy = (correct_count / total) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 100.00%


In [3]:
df=pd.read_csv("IRIS.csv")
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [6]:
df['Species']=df['Species'].replace({'Iris-setosa':1,'Iris-versicolor':0,'Iris-virginica':0})

In [7]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1
3,4,4.6,3.1,1.5,0.2,1
4,5,5.0,3.6,1.4,0.2,1


In [10]:
arr1=np.array([df['SepalLengthCm'],df['SepalWidthCm'],df['PetalLengthCm'],df['PetalWidthCm']]).T
arr2=np.array([df['Species']])
arr1[0:5]
  # Reshape arr2 to have shape (150,)


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [11]:
arr2 = np.array(df['Species']).reshape(-1)
x_train,x_test,y_train,y_test=train_test_split(arr1,arr2,test_size=0.2,random_state=30)

encoder=OneHotEncoder(categories='auto')
y_train_onehot=encoder.fit_transform(y_train.reshape(-1,1)).toarray()

input_size=x_train.shape[1]
hidden_size=8
output_size=y_train_onehot.shape[1]

learning_rate=0.1
epochs=10000
obj_ANN=ANN(input_size, hidden_size, output_size, learning_rate)
obj_ANN.train(x_train,y_train_onehot,epochs)


Epoch 0: Error 0.519740183493933
Epoch 1000: Error 0.007845859027498273
Epoch 2000: Error 0.005509080018937312
Epoch 3000: Error 0.004480752955972447
Epoch 4000: Error 0.003870317452055808
Epoch 5000: Error 0.0034549862636158676
Epoch 6000: Error 0.0031491102095599086
Epoch 7000: Error 0.0029118266770575777
Epoch 8000: Error 0.0027208548582606656
Epoch 9000: Error 0.002562879443197463


In [13]:
for i in range(len(x_test)):
    predicted=obj_ANN.forward_prop(x_test[i])
    predicted_class=np.argmax(predicted)
    actual_class=y_test[i]
    print(f"\t\t{x_test[i]}\t\t    {predicted_class}\t\t     {actual_class}")

		[5.4 3.4 1.7 0.2]		    1		     1
		[5.4 3.9 1.7 0.4]		    1		     1
		[4.6 3.1 1.5 0.2]		    1		     1
		[5.8 2.7 5.1 1.9]		    0		     0
		[5.8 2.7 3.9 1.2]		    0		     0
		[5.5 2.4 3.8 1.1]		    0		     0
		[6.3 2.7 4.9 1.8]		    0		     0
		[6.7 3.  5.2 2.3]		    0		     0
		[6.4 2.9 4.3 1.3]		    0		     0
		[7.3 2.9 6.3 1.8]		    0		     0
		[4.4 3.  1.3 0.2]		    1		     1
		[6.3 2.9 5.6 1.8]		    0		     0
		[5.  2.3 3.3 1. ]		    0		     0
		[5.7 2.8 4.1 1.3]		    0		     0
		[4.4 2.9 1.4 0.2]		    1		     1
		[6.2 2.9 4.3 1.3]		    0		     0
		[5.2 4.1 1.5 0.1]		    1		     1
		[4.9 3.  1.4 0.2]		    1		     1
		[4.8 3.4 1.9 0.2]		    1		     1
		[6.3 2.5 4.9 1.5]		    0		     0
		[6.3 2.8 5.1 1.5]		    0		     0
		[4.4 3.2 1.3 0.2]		    1		     1
		[5.1 3.5 1.4 0.2]		    1		     1
		[5.  3.2 1.2 0.2]		    1		     1
		[7.6 3.  6.6 2.1]		    0		     0
		[6.4 2.8 5.6 2.2]		    0		     0
		[5.9 3.2 4.8 1.8]		    0		     0
		[6.3 3.4 5.6 2.4]		    0		     0
		[4.7 3.2 1.3 0.2]	